In [56]:
import pandas as pd
import numpy as np
from functools import reduce
from pandas.api.types import CategoricalDtype
from datetime import datetime, timedelta
from utils import format_number

from config import LAST_N_DAYS, COL_NAMES_TO_INCLUDE

pd.options.display.float_format = "{:,.2f}".format
pd.set_option("display.max_columns", None)

In [57]:
def extract_source(source_string):
    source_list = source_string.split("-")
    if len(source_list) > 1:
        return source_list[
            1
        ].strip()  # strip() is used to remove any leading/trailing spaces
    else:
        return source_string.strip()


def cleanup_string(source_string):
    return source_string.replace(" ", "").lower()


def remove_brackets(x):
    if isinstance(x, str):
        return x.replace("['", "").replace("']", "")
    else:
        return x


def merge_dfs(key="app_name_join", cols=COL_NAMES_TO_INCLUDE, **dfs):
    df_combined = reduce(
        lambda left, right: pd.merge(left, right, on=key, how="left"), dfs.values()
    )
    return df_combined[cols]


def calculate_metrics(df, op="op_deployed"):
    inc_cols = df.filter(like="incremental_").columns
    inc_cols = [col for col in inc_cols if "per_op" not in col]
    # df = df.assign(**{f'incremental_{col.split("_")[1]}_annualized_per_op': df[col] * 365 / df["net_op_deployed"] for col in inc_cols})
    df = df.assign(
        **{
            f'{col.replace("_per_day", "")}_annualized_per_op': df[col] * 365 / df[op]
            for col in inc_cols
        }
    )
    df["net_tvl_per_op"] = df["cumul_last_price_net_dollar_flow"] / df[op]

    return df

# Incentive Program Summary
Status of programs live, completed and to be announced by season.

In [58]:
df_info = pd.read_csv("inputs/" + "op_incentive_program_info" + ".csv")

# convert to datetime
df_info["start_date"] = pd.to_datetime(
    df_info["Announced On"].fillna(df_info["Start Date"])
)
df_info["end_date"] = pd.to_datetime(df_info["End Date"])

# convert program status into ordered categorical type
cat_size_order = CategoricalDtype(
    ["Live ‎🔥", "Coming soon ‎⏳", "Completed"], ordered=True
)
df_info["Status"] = df_info["Status"].astype(cat_size_order)

# create app_name_join, coalesce with app name override map, app name and remove any space
df_info["app_name_join"] = df_info["App Name Map Override"].fillna(df_info["App Name"])
df_info["app_name_join"] = df_info["app_name_join"].apply(cleanup_string)

In [59]:
for i in ["GovFund", "GovFund Growth Experiments", "All Programs"]:
    # Assign the filters
    if i == "GovFund":
        filter_name = " - GovFund Only"
        df_choice = df_info[df_info["Source"] != "Partner Fund"].copy()
    elif i == "GovFund Growth Experiments":
        filter_name = " - GovFund Growth Exp."
        df_choice = df_info[df_info["Source"] != "Partner Fund"].copy()
        df_choice = df_choice[
            df_choice["Incentive / Growth Program Included?"] == "Yes"
        ]
    else:
        filter_name = ""
        df_choice = df_info.copy()

    # clean up for columns needed
    df_choice = df_choice[
        [
            "Source",
            "Status",
            "# OP Allocated",
            "App Name",
            "start_date",
            "end_date",
            "app_name_join",
            "Incentive / Growth Program Included?",
        ]
    ]
    summary = pd.pivot_table(
        df_choice,
        values=["# OP Allocated", "App Name"],
        index=["Status", "Source"],
        aggfunc={"# OP Allocated": "sum", "App Name": "count"},
    )

    subtotal_name = "Subtotal" + filter_name
    # calculate subtotals on program status
    result = pd.concat(
        [
            summary,
            summary.groupby(level=0)
            .sum()
            .assign(item_name=subtotal_name)
            .set_index("item_name", append=True),
        ]
    ).sort_index(level=[0, 1])
    result = result.sort_index(level=[0, 1], ascending=[True, False])

    # add grand total to summary
    result.loc[("Grand Total"), "# OP Allocated"] = summary["# OP Allocated"].sum()
    result.loc[("Grand Total"), "App Name"] = summary["App Name"].sum()

    # cleanup display
    result["# Programs"] = result["App Name"].astype(int)
    result["# OP Allocated (M)"] = result["# OP Allocated"].apply(format_number)

    # calculate percentage of total
    result.loc[(slice(None), subtotal_name), "# OP Allocated"] / summary[
        "# OP Allocated"
    ].sum()
    result["% OP Allocated"] = (
        round(
            result.loc[(slice(None), subtotal_name), "# OP Allocated"]
            / summary["# OP Allocated"].sum()
            * 100
        )
        .astype(str)
        .replace("\.0", "", regex=True)
        + "%"
    )
    result["% OP Allocated"].fillna("-", inplace=True)

    result = result.replace((0, "0.0M", "0.0"), "-")
    print(i)
    display(result.drop(columns=["# OP Allocated", "App Name"]))
    print()

GovFund


# Programs # OP Allocated (M)  \
Status         Source                                                  
Live ‎🔥        Subtotal - GovFund Only         35              41.2M   
               Governance - Season 3            -                  -   
               Governance - Season 2           12               7.2M   
               Governance - Season 1            9               4.5M   
               Governance - Phase 0            14              29.5M   
Coming soon ‎⏳ Subtotal - GovFund Only         54              14.4M   
               Governance - Season 3           22               2.5M   
               Governance - Season 2           18               5.7M   
               Governance - Season 1            7               2.1M   
               Governance - Phase 0             7               4.1M   
Completed      Subtotal - GovFund Only          5               2.6M   
               Governance - Season 3            -                  -   
               Governance - Season 2            1             240.0K   
               Governance - Season 1            -                  -   
               Governance - Phase 0             4               2.4M   
Grand Total                                    94              58.2M   

                                       % OP Allocated  
Status         Source                                  
Live ‎🔥        Subtotal - GovFund Only            71%  
               Governance - Season 3                -  
               Governance - Season 2                -  
               Governance - Season 1                -  
               Governance - Phase 0                 -  
Coming soon ‎⏳ Subtotal - GovFund Only            25%  
               Governance - Season 3                -  
               Governance - Season 2                -  
               Governance - Season 1                -  
               Governance - Phase 0                 -  
Completed      Subtotal - GovFund Only             5%  
               Governance - Season 3                -  
               Governance - Season 2                -  
               Governance - Season 1                -  
               Governance - Phase 0                 -  
Grand Total                                         -


GovFund Growth Experiments


# Programs # OP Allocated (M)  \
Status         Source                                                         
Live ‎🔥        Subtotal - GovFund Growth Exp.         33              41.1M   
               Governance - Season 3                   -                  -   
               Governance - Season 2                  10               7.1M   
               Governance - Season 1                   9               4.5M   
               Governance - Phase 0                   14              29.5M   
Coming soon ‎⏳ Subtotal - GovFund Growth Exp.         32              10.9M   
               Governance - Season 3                  12               2.1M   
               Governance - Season 2                  13               5.2M   
               Governance - Season 1                   4               1.3M   
               Governance - Phase 0                    3               2.2M   
Completed      Subtotal - GovFund Growth Exp.          5               2.6M   
               Governance - Season 3                   -                  -   
               Governance - Season 2                   1             240.0K   
               Governance - Season 1                   -                  -   
               Governance - Phase 0                    4               2.4M   
Grand Total                                           70              54.6M   

                                              % OP Allocated  
Status         Source                                         
Live ‎🔥        Subtotal - GovFund Growth Exp.            75%  
               Governance - Season 3                       -  
               Governance - Season 2                       -  
               Governance - Season 1                       -  
               Governance - Phase 0                        -  
Coming soon ‎⏳ Subtotal - GovFund Growth Exp.            20%  
               Governance - Season 3                       -  
               Governance - Season 2                       -  
               Governance - Season 1                       -  
               Governance - Phase 0                        -  
Completed      Subtotal - GovFund Growth Exp.             5%  
               Governance - Season 3                       -  
               Governance - Season 2                       -  
               Governance - Season 1                       -  
               Governance - Phase 0                        -  
Grand Total                                                -


All Programs


# Programs # OP Allocated (M)  \
Status         Source                                                
Live ‎🔥        Subtotal                      35              41.2M   
               Partner Fund                   -                  -   
               Governance - Season 3          -                  -   
               Governance - Season 2         12               7.2M   
               Governance - Season 1          9               4.5M   
               Governance - Phase 0          14              29.5M   
Coming soon ‎⏳ Subtotal                      54              14.4M   
               Partner Fund                   -                  -   
               Governance - Season 3         22               2.5M   
               Governance - Season 2         18               5.7M   
               Governance - Season 1          7               2.1M   
               Governance - Phase 0           7               4.1M   
Completed      Subtotal                       8              11.1M   
               Partner Fund                   3               8.4M   
               Governance - Season 3          -                  -   
               Governance - Season 2          1             240.0K   
               Governance - Season 1          -                  -   
               Governance - Phase 0           4               2.4M   
Grand Total                                  97              66.7M   

                                     % OP Allocated  
Status         Source                                
Live ‎🔥        Subtotal                         62%  
               Partner Fund                       -  
               Governance - Season 3              -  
               Governance - Season 2              -  
               Governance - Season 1              -  
               Governance - Phase 0               -  
Coming soon ‎⏳ Subtotal                         22%  
               Partner Fund                       -  
               Governance - Season 3              -  
               Governance - Season 2              -  
               Governance - Season 1              -  
               Governance - Phase 0               -  
Completed      Subtotal                         17%  
               Partner Fund                       -  
               Governance - Season 3              -  
               Governance - Season 2              -  
               Governance - Season 1              -  
               Governance - Phase 0               -  
Grand Total                                       -

In [60]:
# display new programs in last 30 days
df_new_programs = df_choice[
    df_choice["start_date"] > pd.Timestamp("today") - timedelta(days=LAST_N_DAYS)
].sort_values(by="start_date", ascending=False)
if not df_new_programs.empty:
    df_new_programs["end_date"].fillna("-", inplace=True)
    display(df_new_programs.drop("app_name_join", axis=1))

,Source,Status,# OP Allocated,App Name,start_date,end_date,Incentive / Growth Program Included?
67,NaN,Live ‎🔥,NaN,Quests on Coinbase Wallet,2023-03-09,-,NaN
81,Governance - Season 2,Live ‎🔥,"504,000.00",Sushiswap,2023-03-03,-,Yes
11,Governance - Season 2,Live ‎🔥,"33,000.00",Bankless Academy,2023-02-22,-,No - Grants Only
8,Governance - Season 2,Live ‎🔥,"250,000.00",Angle,2023-02-20,-,Yes
93,Governance - Season 1,Live ‎🔥,"300,000.00",WardenSwap,2023-02-17,-,Yes


In [61]:
# display completed programs in last 30 days
df_completed = df_choice[
    (df_choice["Status"] == "Completed")
    & (df_choice["end_date"] > pd.Timestamp("today") - timedelta(days=LAST_N_DAYS))
].sort_values(by="start_date", ascending=False)
if not df_completed.empty:
    display(df_completed.drop("app_name_join", axis=1))

,Source,Status,# OP Allocated,App Name,start_date,end_date,Incentive / Growth Program Included?
69,Governance - Season 2,Completed,"240,000.00",Revert Finance,2022-11-03,2023-03-06,Yes


# Usage and TVL Attribution
To combine all sources of data together

In [62]:
# read in input data
df_usage = pd.read_csv("csv_outputs/" + "dune_op_program_performance_summary" + ".csv")
# convert to datetime
df_usage["start_date"] = pd.to_datetime(df_usage["start_date"])
df_usage["end_date"] = pd.to_datetime(df_usage["end_date"])

df_usage["app_name_join"] = df_usage["app_name"].apply(cleanup_string)
df_usage["duration_days"] = (
    df_usage["end_date"].fillna(datetime.now()) - df_usage["start_date"]
).dt.days + 1  # if start and end date is the same, add 1 to include that day


df_tvl = pd.read_csv("csv_outputs/op_summer_latest_stats.csv")
df_tvl = df_tvl[df_tvl["include_in_summary"] == 1]
df_tvl["app_name_join"] = df_tvl["parent_protocol"].apply(cleanup_string)

df_op_distribution = pd.read_csv("csv_outputs/dune_op_distribution_type.csv")
df_op_distribution["net_op_deployed"] = (
    df_op_distribution["op_deployed"] - df_op_distribution["op_from_other_projects"]
).astype(float)
df_op_distribution["app_name_join"] = df_op_distribution["from_name"].apply(
    cleanup_string
)

# filter to incentive / growth programs only
df_choice = df_choice[df_choice["Incentive / Growth Program Included?"] == "Yes"]

In [63]:
df_to_summarize = {
    # df | groupby | column to summarize
    "df_choice": ("app_name_join", "# OP Allocated"),
    "df_tvl": (
        "app_name_join",
        [
            "cumul_last_price_net_dollar_flow",
            "cumul_last_price_net_dollar_flow_at_program_end",
        ],
    ),
    "df_op_distribution": ("app_name_join", ["op_deployed", "net_op_deployed"]),
}

summary_dfs = {}  # create an empty dictionary to store the resulting DataFrames

for df_name, (groupby_col, sum_cols) in df_to_summarize.items():
    df = globals()[df_name]  # assuming the dataframes are stored as global variables
    if isinstance(sum_cols, str):  # if only one column to sum is specified
        sum_cols = [sum_cols]
    # groupby and sum the specified columns
    grouped = df.groupby(groupby_col)[sum_cols].sum().reset_index()
    # create a new variable with the summary DataFrame
    first_word = groupby_col.split("_")[0]
    summary_df_name = f"{df_name}_summary_{first_word}"
    summary_dfs[summary_df_name] = grouped

# unpack summary_dfs into separate variables with the same names
locals().update(summary_dfs)

# access each summary DataFrame by its variable name
# df_choice_summary_app
# df_tvl_summary_app
# df_op_distribution_summary_app

### By App

In [74]:
# by app
df_combined_app = merge_dfs(
    df_usage=df_usage,
    df_tvl_summary_app=df_tvl_summary_app,
    df_choice_summary_app=df_choice_summary_app,
    df_op_distribution_summary_app=df_op_distribution_summary_app,
)

# calculate metrics
result_app = calculate_metrics(
    df_combined_app, op="op_deployed"
)  # by app use op_deployed
# display(result_app)

In [65]:
# sort by tvl
cols = [
    "app_name",
    "# OP Allocated",
    "net_op_deployed",
    "cumul_last_price_net_dollar_flow",
    "net_tvl_per_op",
]
display(
    result_app[cols]
    .sort_values("cumul_last_price_net_dollar_flow", ascending=False)
    .reset_index(drop=True)
    .head(10)
)

,app_name,# OP Allocated,net_op_deployed,cumul_last_price_net_dollar_flow,net_tvl_per_op
0,Velodrome,"7,000,000.00","5,054,767.00","201,955,598.20",39.58
1,Synthetix,"9,000,000.00","4,919,425.00","83,886,839.97",17.05
2,Aave,"5,000,000.00","4,820,781.00","77,329,625.73",16.04
3,Rocket Pool,"600,000.00","205,002.00","36,404,537.59",177.58
4,Beefy Finance,"650,000.00","281,962.00","34,600,187.71",122.71
5,Stargate Finance,"1,000,000.00","469,725.00","26,951,682.20",57.38
6,Beethoven X,"500,000.00","164,703.00","26,250,099.00",123.94
7,PoolTogether,"1,000,000.00","842,488.00","25,209,238.41",29.92
8,Pika Protocol,"900,000.00","672,632.00","10,999,054.19",16.35
9,dHedge,"350,000.00","202,257.00","8,728,060.05",43.15


In [72]:
# sort by txs
txs_cols = [
    "app_name",
    "# OP Allocated",
    "net_op_deployed",
    "incremental_txs_per_day",
    "incremental_txs_annualized_per_op",
    "incremental_txs_per_day_after",
    "incremental_txs_after_annualized_per_op",
]

# # result_app[txs_cols].to_csv('csv_outputs/transaction_stats_by_app.csv')

display(
    result_app[txs_cols]
    .sort_values("incremental_txs_per_day", ascending=False)
    .reset_index(drop=True)
    .head(10)
)

display(
    result_app[txs_cols]
    .sort_values("incremental_txs_after_annualized_per_op", ascending=False)
    .dropna()
    .reset_index(drop=True)
    .head(10)
)

,app_name,# OP Allocated,net_op_deployed,incremental_txs_per_day,incremental_txs_annualized_per_op,incremental_txs_per_day_after,incremental_txs_after_annualized_per_op
0,Velodrome,"7,000,000.00","5,054,767.00","8,045.35",0.5756,NaN,nan
1,Uniswap,"1,000,000.00","150,001.00","5,666.34",13.7880,NaN,nan
2,Pika Protocol,"900,000.00","672,632.00","4,781.95",2.5949,NaN,nan
3,Rubicon,"1,100,000.00","791,082.00","4,110.26",1.8964,"1,583.82",0.7308
4,Synthetix,"9,000,000.00","4,919,425.00","4,092.16",0.3036,NaN,nan
5,Aave,"5,000,000.00","4,820,781.00","3,122.79",0.2364,"4,744.49",0.3592
6,Hop Protocol,"1,000,000.00","152,602.00","3,003.30",7.1834,NaN,nan
7,Beethoven X,"500,000.00","164,703.00","2,297.19",3.9589,NaN,nan
8,1inch,"300,000.00","300,000.00","2,100.50",2.5556,390.16,0.4747
9,PoolTogether,"1,000,000.00","842,488.00","1,910.07",0.8275,NaN,nan


,app_name,# OP Allocated,net_op_deployed,incremental_txs_per_day,incremental_txs_annualized_per_op,incremental_txs_per_day_after,incremental_txs_after_annualized_per_op
0,Rubicon,"1,100,000.00","791,082.00","4,110.26",1.8964,"1,583.82",0.7308
1,1inch,"300,000.00","300,000.00","2,100.50",2.5556,390.16,0.4747
2,Revert Finance,"240,000.00","240,839.00",217.56,0.3297,246.67,0.3738
3,Aave,"5,000,000.00","4,820,781.00","3,122.79",0.2364,"4,744.49",0.3592
4,WePiggy,"300,000.00","300,002.00",39.35,0.0479,12.07,0.0147
5,Aelin,"900,000.00","900,002.00",7.83,0.0032,-4.99,-0.0020


In [75]:
# sort by gas
gas_cols = [
    "app_name",
    "# OP Allocated",
    "net_op_deployed",
    "incremental_gas_fee_eth_per_day",
    "incremental_gas_fee_eth_annualized_per_op",
    "incremental_gas_fee_eth_per_day_after",
    "incremental_gas_fee_eth_after_annualized_per_op",
]

result_app.loc[
    :, result_app.columns.str.contains("annualized_per_op")
] = result_app.loc[:, result_app.columns.str.contains("annualized_per_op")].applymap(
    "{:.4f}".format
)

display(
    result_app[gas_cols]
    .sort_values("incremental_gas_fee_eth_per_day", ascending=False)
    .reset_index(drop=True)
    .head(10)
)

display(
    result_app[gas_cols]
    .sort_values("incremental_gas_fee_eth_after_annualized_per_op", ascending=False)
    .dropna()
    .reset_index(drop=True)
    .head(10)
)

,app_name,# OP Allocated,net_op_deployed,incremental_gas_fee_eth_per_day,incremental_gas_fee_eth_annualized_per_op,incremental_gas_fee_eth_per_day_after,incremental_gas_fee_eth_after_annualized_per_op
0,Synthetix,"9,000,000.00","4,919,425.00",55.36,0.0041,NaN,nan
1,Velodrome,"7,000,000.00","5,054,767.00",19.45,0.0014,NaN,nan
2,Hop Protocol,"1,000,000.00","152,602.00",10.36,0.0248,NaN,nan
3,Uniswap,"1,000,000.00","150,001.00",5.63,0.0137,NaN,nan
4,Beethoven X,"500,000.00","164,703.00",4.64,0.0080,NaN,nan
5,Aave,"5,000,000.00","4,820,781.00",4.15,0.0003,7.47,0.0006
6,Rubicon,"1,100,000.00","791,082.00",3.96,0.0018,5.17,0.0024
7,QiDao,"750,000.00","342,865.00",3.48,0.0037,NaN,nan
8,Stargate Finance,"1,000,000.00","469,725.00",2.90,0.0023,NaN,nan
9,1inch,"300,000.00","300,000.00",2.74,0.0033,-0.03,-0.0000


,app_name,# OP Allocated,net_op_deployed,incremental_gas_fee_eth_per_day,incremental_gas_fee_eth_annualized_per_op,incremental_gas_fee_eth_per_day_after,incremental_gas_fee_eth_after_annualized_per_op
0,Rubicon,"1,100,000.00","791,082.00",3.96,0.0018,5.17,0.0024
1,Aave,"5,000,000.00","4,820,781.00",4.15,0.0003,7.47,0.0006
2,Revert Finance,"240,000.00","240,839.00",0.06,0.0001,0.16,0.0002
3,WePiggy,"300,000.00","300,002.00",0.08,0.0001,0.08,0.0001
4,1inch,"300,000.00","300,000.00",2.74,0.0033,-0.03,-0.0000
5,Aelin,"900,000.00","900,002.00",0.03,0.0000,-0.04,-0.0000


### By Fund Source

In [76]:
agg_dict = {
    "# OP Allocated": "sum",
    "net_op_deployed": "sum",
    # "incremental_addr_per_day": "sum",
    "incremental_txs_per_day": "sum",
    "incremental_gas_fee_eth_per_day": "sum",
    "incremental_txs_per_day_after": "sum",
    # "incremental_addr_per_day_after": "sum",
    "incremental_gas_fee_eth_per_day_after": "sum",
    "cumul_last_price_net_dollar_flow": "sum",
}

In [77]:
result_app["op_source_length"] = result_app["op_source"].str.split(",").apply(len)
result_app["op_source_map"] = np.where(
    result_app["op_source_length"] > 1, ["Multiple"], result_app["op_source"]
)

# single programs
result_source = result_app.groupby("op_source_map").agg(agg_dict)

# calculate metrics
result_source = calculate_metrics(
    result_source, op="net_op_deployed"
)  # use net to avoid double counting
result_source = result_source.reset_index()
result_source["op_source_map"] = result_source["op_source_map"].apply(
    lambda x: remove_brackets(x)
)
result_source.sort_values("op_source_map").reset_index()

display(result_source)

,op_source_map,# OP Allocated,net_op_deployed,incremental_txs_per_day,incremental_gas_fee_eth_per_day,incremental_txs_per_day_after,incremental_gas_fee_eth_per_day_after,cumul_last_price_net_dollar_flow,incremental_txs_annualized_per_op,incremental_gas_fee_eth_annualized_per_op,incremental_txs_after_annualized_per_op,incremental_gas_fee_eth_after_annualized_per_op,net_tvl_per_op
0,Multiple,"9,450,000.00","6,946,246.00","10,700.99",20.22,0.00,0.00,"227,042,064.86",0.56,0.00,0.00,0.00,32.69
1,Partner Fund,"5,000,000.00","4,820,781.00","4,125.73",4.34,"4,744.49",7.47,"77,329,625.73",0.31,0.00,0.36,0.00,16.04
2,Phase 0,"31,200,000.00","16,834,319.00","16,775.45",74.79,"1,981.04",5.18,"128,107,107.90",0.36,0.00,0.04,0.00,7.61
3,Season 1,"4,500,000.00","2,048,668.00","3,378.48",10.19,0.00,0.00,"111,565,196.03",0.60,0.00,0.00,0.00,54.46
4,Season 2,"2,464,069.00","978,022.00","1,213.58",3.11,246.67,0.16,"16,850,062.98",0.45,0.00,0.09,0.00,17.23


In [82]:
# remove any non growth projects from usage
# find out why the by source number is not correct also make it growth only!

result_app.groupby("op_source_map").sum()["# OP Allocated"].reset_index()

,op_source_map,# OP Allocated
0,Multiple,"9,450,000.00"
1,['Partner Fund'],"5,000,000.00"
2,['Phase 0'],"31,200,000.00"
3,['Season 1'],"4,500,000.00"
4,['Season 2'],"2,464,069.00"
